In [1]:

include("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh
using JSON
using Pkg
Pkg.add("Plots")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
data = JSON.parsefile("test.json")
Systems_params= data["Systems"]
mesh_params= data["Mesh"]

α= α3b(0.5,0.5,1,Systems_params["lmax"],Systems_params["lmin"],Systems_params["λmax"],Systems_params["λmin"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Systems_params["MT"])

grid= initialmesh(mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"],Float64(mesh_params["xmax"]),Float64(mesh_params["ymax"]),Float64(mesh_params["alpha"]))

In [ ]:
for i in 1:mesh_params["nx"]
    println(grid.xi[i], "   ", grid.dxi[i])
end

In [ ]:
using Revise
Revise.revise()
includet("Gcoefficient.jl")
using .Gcoefficient

initialY(λmax, lmax, nθ, nx, ny, cosθi, xi, yi, P::Char)
return Yλαout, Ylαin, Yλαin

Yλin[iθ, iy, ix, nch, perm_index]

In [ ]:
G1=computeGcoffecient(α, grid)

In [ ]:
using Revise
includet("twobody.jl")
using .twobodybound

In [ ]:
potname="AV18"
bound_states=bound2b(grid, potname)

In [2]:
using Revise
includet("spline.jl")
using .spline

In [3]:
"""
Test the cubic Hermitian spline interpolation with a polynomial function.
"""
# Import Printf for formatting
using Printf
using Plots
function test_cubherm_interpolation()
    # Define the polynomial function f(x) = 5x^5 + 4x^4 + 3x^3 + 2x^2 + x
    f(x) = 5*x^5 + 4*x^4 + 3*x^3 + 2*x^2 + x
    
    # Create a set of old grid points (coarse grid)
    xold = collect(range(-1.0, 1.0, length=15))
    
    # Evaluate the function at the old grid points
    yold = f.(xold)
    
    # println("Original grid points:")
    # for i in 1:length(xold)
    #     println("x = $(xold[i]), f(x) = $(yold[i])")
    # end
    # println()
    
    # Create a set of test points (fine grid) where we want to interpolate
    xtest = collect(range(-0.95, 0.95, length=20))
    
    # Compute the exact values
    yexact = f.(xtest)
    
    # Interpolate using our cubic Hermitian spline
    yinterp = Float64[]
    for x in xtest
        push!(yinterp, cubherm_interp_point(xold, yold, x))
    end
    
    # Compute errors
    errors = abs.(yinterp - yexact)
    max_error = maximum(errors)
    avg_error = sum(errors) / length(errors)
    
    println("Interpolation results and errors:")
    println("   x       Exact        Interpolated       Error")
    println("------------------------------------------------------")
    for i in 1:length(xtest)
        # Use simple string formatting instead of @printf/@sprintf
        x_str = @sprintf("%.4f", xtest[i])
        exact_str = @sprintf("%.6f", yexact[i])
        interp_str = @sprintf("%.6f", yinterp[i])
        error_str = @sprintf("%.6e", errors[i])
        
        println("$(lpad(x_str, 7))  $(lpad(exact_str, 12))  $(lpad(interp_str, 12))  $(lpad(error_str, 12))")
    end
    
    println("\nMaximum error: $max_error")
    println("Average error: $avg_error")
    
    # Optional: Plot the results if Plots.jl is available
    try
        
        
        p = plot(xold, yold, seriestype=:scatter, 
             label="Original Points", markersize=6, legend=:topleft)
        
        plot!(p, xtest, yexact, 
              label="Exact Function", linewidth=2)
        
        plot!(p, xtest, yinterp, 
              label="Interpolated", linewidth=2, linestyle=:dash)
        
        # Plot error
        p_error = plot(xtest, errors, 
                   label="Interpolation Error", linewidth=2,
                   xlabel="x", ylabel="Error", title="Interpolation Error")
        
        # Display both plots
        plot(p, p_error, layout=(2,1), size=(800, 600),
             title=["Cubic Hermitian Spline Interpolation" ""])
        
        savefig("interpolation_test.png")
        println("\nPlot saved as 'interpolation_test.png'")
    catch
        println("\nPlotting skipped (Plots.jl not available)")
    end
    
    return max_error, avg_error
end

test_cubherm_interpolation (generic function with 1 method)

In [4]:
using Printf 
test_cubherm_interpolation()

Interpolation results and errors:
   x       Exact        Interpolated       Error
------------------------------------------------------
-0.9500     -2.328005     -2.450916  1.229116e-01
-0.8500     -1.377877     -1.374544  3.332682e-03
-0.7500     -0.811523     -0.814772  3.248681e-03
-0.6500     -0.494995     -0.491207  3.788643e-03
-0.5500     -0.329742     -0.326736  3.006452e-03
-0.4500     -0.246614     -0.247700  1.085854e-03
-0.3500     -0.199861     -0.199010  8.508227e-04
-0.2500     -0.161133     -0.160290  8.424573e-04
-0.1500     -0.113480     -0.113702  2.225794e-04
-0.0500     -0.045352     -0.046313  9.610156e-04
 0.0500      0.055402      0.055129  2.726579e-04
 0.1500      0.207530      0.208283  7.531708e-04
 0.2500      0.442383      0.438300  4.083294e-03
 0.3500      0.809911      0.808031  1.879582e-03
 0.4500      1.384664      1.389009  4.345309e-03
 0.5500      2.271792      2.263719  8.073529e-03
 0.6500      3.613045      3.605919  7.126439e-03
 0.7500     

(0.34594358702847217, 0.026413649084087255)